<a href="https://colab.research.google.com/github/rogersnet/mec-mini-projects/blob/master/mec-3.4.1-api-mini-project/api_data_wrangling_mini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

myyfLzxDgtXsbMQjGucj


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [10]:
# First, import the relevant modules
import requests
import json
from collections import defaultdict

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [12]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
params = {'api_key': API_KEY}
url_path = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json'
nasdaq_r = requests.get(url_path, params=params)

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
nasdaq_r.json()

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [ ]:
# collect data for AFX_X for the year
params = {'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date':'2017-12-31'}
afx_x_stock_data = requests.get(url_path, params=params)

In [20]:
# convert the returned json into a dictionary
afx_x_stock_data_dict = json.loads(afx_x_stock_data.text)

In [42]:
opening_price_col_idx = afx_x_stock_data_dict['dataset']['column_names'].index('Open')

# list of opening prices
opening_prices_list = [x[opening_price_col_idx] for x in afx_x_stock_data_dict['dataset']['data'] if x[opening_price_col_idx] != None]
opening_prices_list

# highest opening price for the stock during this period
highest_opening_price = max(opening_prices_list)
print("Highest Opening price for the stock during this period : {} ".format(highest_opening_price))

# lowest opening price for the stock during this period
lowest_opening_price = min(opening_prices_list)
print("Lowest Opening price for the stock during this period : {} ".format(lowest_opening_price))

Highest Opening price for the stock during this period : 53.11 
Lowest Opening price for the stock during this period : 34.0 


In [52]:
high_price_col_idx = afx_x_stock_data_dict['dataset']['column_names'].index('High')
low_price_col_idx = afx_x_stock_data_dict['dataset']['column_names'].index('Low')

# largest change in any one day
largest_change_single_day = max([x[high_price_col_idx] - x[low_price_col_idx] for x in afx_x_stock_data_dict['dataset']['data']])
print("Largest change in a single day : {:.2f}".format(largest_change_single_day))

Largest change in a single day : 2.81


In [58]:
# function to compute the max difference between any two elements
def max_diff(price_list):
  max_d = price_list[1] - price_list[0]

  for i in range(0, len(price_list)):
    for j in range(i+1, len(price_list)):
      if price_list[j] - price_list[i] > max_d:
        max_d = price_list[j] - price_list[i]

  return max_d

closing_price_idx = afx_x_stock_data_dict['dataset']['column_names'].index('Close')
closing_prices_list = [x[closing_price_idx] for x in afx_x_stock_data_dict['dataset']['data'] if x[closing_price_idx] != None]

# Largest change between any two days (based on Closing Price)
print('Largest change between any two days: {:.2f}'.format(max_diff(closing_prices_list)))

Largest change between any two days: 5.98


In [63]:
# function to compute the median of a list
def median(lst):
    sorted_lst = sorted(lst) 
    if len(sorted_lst) % 2 == 0:  
        return (sorted_lst[len(sorted_lst) // 2] + sorted_lst[(len(sorted_lst) - 1) // 2]) / 2
    else:
        return sorted_lst[len(sorted_lst) // 2]

traded_volume_idx = afx_x_stock_data_dict['dataset']['column_names'].index('Traded Volume')
traded_volume_list = [x[traded_volume_idx] for x in afx_x_stock_data_dict['dataset']['data'] if x[traded_volume_idx] != None]

# Average daily trading volume during the year
print('Average trading volume for the year: {:.2f}'.format(sum(traded_volume_list)/len(traded_volume_list)))

# Median trading volume during the year
print('Median trading volumne for the year: {:.2f}'.format(median(traded_volume_list)))

Average trading volume for the year: 89124.34
Median trading volumne for the year: 76286.00
